In [ ]:
from pathlib import Path
import pysam


In [ ]:
infiles = [Path(infile) for infile in snakemake.input]
print(snakemake.output)
cov_5_perc = str(snakemake.output[0])
cov_1_perc = str(snakemake.output[1])
count_5_perc = str(snakemake.output[2])
count_1_perc = str(snakemake.output[3])

In [ ]:
import numpy as np
import pandas as pd
accession_list_chr = [
    "CM044164.1", "CM044165.1", "CM044166.1", "CM044167.1",
    "CM044168.1", "CM044169.1", "CM044170.1", "CM044171.1",
    "CM044172.1", "CM044173.1", "CM044174.1", "CM044175.1",
    "CM044176.1"
]

First, we create the index:

In [ ]:
for infile in infiles:
    pysam.index(str(infile))

Now, we open the bam file:

In [ ]:
samfiles = [pysam.AlignmentFile(infile, "rb") for infile in infiles]

## Number of mapped and unmapped reads

In [ ]:
print(f"There are {samfiles[0].mapped:,.0f} mapped reads after filtering best 5%")
print(f"There are {samfiles[0].unmapped:,.0f} unmapped reads after filtering best 5%")
print(f"{samfiles[0].mapped/(samfiles[0].unmapped + samfiles[0].mapped)*100:,.2f}% were mapped after filtering best 5%")

print(f"There are {samfiles[1].mapped:,.0f} mapped reads after filtering best 1%")
print(f"There are {samfiles[1].unmapped:,.0f} unmapped reads after filtering best 1%")
print(f"{samfiles[1].mapped/(samfiles[1].unmapped + samfiles[1].mapped)*100:,.2f}% were mapped after filtering best 1%")

In [ ]:
print(f"There are {samfiles[0].nreferences} contigs")
print(f"There are {sum(samfiles[0].lengths[samfiles[0].get_tid(chr)] for chr in accession_list_chr):,.0f} nucleotides in chromosomes in the reference genome")

## Summarize coverage
Now, we are going to create 2 csv tables for summarize coverage and mapped reads to chr 

In [ ]:
from functools import reduce
def count_coverage(samfile: pysam.AlignmentFile, contig: str)-> int:
    counts_per_each_base = samfile.count_coverage(
        contig=contig, quality_threshold=0
    )
    add_ = lambda x, y: np.add(x, y, dtype='int64')
    return reduce(add_, counts_per_each_base)

In [ ]:
def summarise_coverage(samfile):
    coverages_per_base = [count_coverage(samfile, chr) for chr in accession_list_chr]
    contigs_stats = samfile.get_index_statistics()
    return pd.DataFrame({
        'contig': samfile.references[0:13],
        'chr_length': samfile.lengths[0:13],
        'mapped_reads': [contig.mapped for contig in contigs_stats[0:13]],
        'reads_length': [sum([x.infer_read_length() for x in samfile.fetch(chr)]) for chr in accession_list_chr],
        'unmapped_reads': [contig.unmapped for contig in contigs_stats[0:13]],
        'coverage': [np.sum(x) for x in coverages_per_base],
        })
    

In [ ]:
df = summarise_coverage(samfiles[0])
df.to_csv(cov_5_perc, index=None)
df

In [ ]:
df = summarise_coverage(samfiles[1])
df.to_csv(cov_1_perc, index=None)
df

In [ ]:
def summarize_count(samfile):
    coverages_per_base = [count_coverage(samfile, chr) for chr in accession_list_chr]
    return pd.DataFrame({
        'contig': np.concatenate([(np.amax(x)+1)*[chr] for x, chr in zip(coverages_per_base, accession_list_chr)]),
        'coverage_bin': np.concatenate([list(range(0, np.amax(x)+1)) for x in coverages_per_base]),
        'counts': np.concatenate([np.bincount(x) for x in coverages_per_base])
        })

In [ ]:
df = summarize_count(samfiles[0])
df.to_csv(count_5_perc, index=None)
df

In [ ]:
df = summarize_count(samfiles[1])
df.to_csv(count_1_perc, index=None)
df